<a href="https://colab.research.google.com/github/ENEE439d-TEXTML/TextML/blob/master/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data  = pd.read_csv("/content/drive/MyDrive/UMD - senior year/spring 2022/439D/project/data.csv")

# exmaple code : 
https://towardsdatascience.com/word-embeddings-for-sentiment-analysis-65f42ea5d26e

## Setup necessary libraries
> the imports and stuff

In [ ]:
#import all necessary libraries for this tutorial

import pandas as pd
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
import keras

In [ ]:
#parameters that will be used in tutorial

NB_WORDS = 10000  # Parameter indicating the number of words we'll put in the dictionary
VAL_SIZE = 1000  # Size of the validation set
NB_START_EPOCHS = 10  # Number of epochs we usually start to train with
BATCH_SIZE = 512  # Size of the batches used in the mini-batch gradient descent
MAX_LEN = 24  # Maximum number of words in a sequence
GLOVE_DIM = 100  # Number of dimensions of the GloVe word embeddings


In [ ]:
data.head()

,no,paper,cited_paper,label,text
0,0,A00-1043,A00-2024,0,We analyzed a set of articles and identified s...
1,1,H05-1033,A00-2024,0,Table 3: Example compressions Compression AvgL...
2,2,I05-2009,A00-2024,0,5.3 Related works and discussion Our two-step ...
3,3,I05-2009,A00-2024,0,(1999) proposed a summarization system based o...
4,4,I05-2009,A00-2024,0,We found that the deletion of lead parts did n...


## Removing non-alphabetic characters
> remove words and characters that aren't useful from sentences in dataset

In [ ]:
#define functions to things with no sentiment value (irrelevant words)
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

characters = [""]

#function to remove stopwords
def remove_stopwords(data):
  data['review_without_stopwords'] = data['text'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

#function to remove non-alphabetical tags
def remove_tags(string):
    #result = re.sub('[\d<.*?:>()-,;|/@!#$%^&*~`_=+]','',string)

    pattern = re.compile('[\W_0-9]+')
    dirty_list = string.split()
    clean_list = [pattern.sub('', word) for word in dirty_list]
    result = ' '.join(clean_list)
    
    # result = re.sub('[\W_0-9]+','',string)    #see https://blog.finxter.com/how-to-remove-all-non-alphabet-characters-from-a-string/ for explanation
    # result = re.sub('  ',' ',result)
    return result

In [ ]:
#remove stopwords
clean_dataset = remove_stopwords(data)
clean_dataset['w/o stopwords or tags']= clean_dataset['review_without_stopwords'].apply(lambda cw : remove_tags(cw))
clean_dataset

,no,paper,cited_paper,label,text,review_without_stopwords,w/o stopwords or tags
0,0,A00-1043,A00-2024,0,We analyzed a set of articles and identified s...,We analyzed set articles identified six major ...,We analyzed set articles identified six major ...
1,1,H05-1033,A00-2024,0,Table 3: Example compressions Compression AvgL...,Table 3: Example compressions Compression AvgL...,Table Example compressions Compression AvgLen...
2,2,I05-2009,A00-2024,0,5.3 Related works and discussion Our two-step ...,5.3 Related works discussion Our two-step mode...,Related works discussion Our twostep model es...
3,3,I05-2009,A00-2024,0,(1999) proposed a summarization system based o...,(1999) proposed summarization system based dra...,proposed summarization system based draft rev...
4,4,I05-2009,A00-2024,0,We found that the deletion of lead parts did n...,We found deletion lead parts not occur often s...,We found deletion lead parts not occur often s...
...,...,...,...,...,...,...,...
994,994,N09-1053,J92-4003,0,While we can only compare class models with wo...,While can compare class models word models lar...,While can compare class models word models lar...
995,995,P01-1046,J92-4003,0,(1999) and Lee (1999)) can be generally divide...,(1999) Lee (1999)) can generally divided three...,Lee can generally divided three types discou...
996,996,P01-1046,J92-4003,0,Classes can be induced directly from the corpu...,Classes can induced directly corpus (Pereira e...,Classes can induced directly corpus Pereira et...
997,997,P01-1068,J92-4003,0,And we consider that word pairs that have a sm...,And consider word pairs small distance vectors...,And consider word pairs small distance vectors...


In [ ]:
clean_dataset

,no,paper,cited_paper,label,text,review_without_stopwords,w/o stopwords or tags
0,0,A00-1043,A00-2024,0,We analyzed a set of articles and identified s...,We analyzed set articles identified six major ...,We analyzed set articles identified six major ...
1,1,H05-1033,A00-2024,0,Table 3: Example compressions Compression AvgL...,Table 3: Example compressions Compression AvgL...,Table Example compressions Compression AvgLen...
2,2,I05-2009,A00-2024,0,5.3 Related works and discussion Our two-step ...,5.3 Related works discussion Our two-step mode...,Related works discussion Our twostep model es...
3,3,I05-2009,A00-2024,0,(1999) proposed a summarization system based o...,(1999) proposed summarization system based dra...,proposed summarization system based draft rev...
4,4,I05-2009,A00-2024,0,We found that the deletion of lead parts did n...,We found deletion lead parts not occur often s...,We found deletion lead parts not occur often s...
...,...,...,...,...,...,...,...
994,994,N09-1053,J92-4003,0,While we can only compare class models with wo...,While can compare class models word models lar...,While can compare class models word models lar...
995,995,P01-1046,J92-4003,0,(1999) and Lee (1999)) can be generally divide...,(1999) Lee (1999)) can generally divided three...,Lee can generally divided three types discou...
996,996,P01-1046,J92-4003,0,Classes can be induced directly from the corpu...,Classes can induced directly corpus (Pereira e...,Classes can induced directly corpus Pereira et...
997,997,P01-1068,J92-4003,0,And we consider that word pairs that have a sm...,And consider word pairs small distance vectors...,And consider word pairs small distance vectors...


## Remove data that is too large

In [ ]:
# get maxLen for the maximum length of a text 
maxLen = max(clean_dataset['w/o stopwords or tags'].apply(len))

#the row of the maxLen text
clean_dataset.loc[clean_dataset['w/o stopwords or tags'].apply(len) == max(clean_dataset['w/o stopwords or tags'].apply(len))]

,no,paper,cited_paper,label,text,review_without_stopwords,w/o stopwords or tags
225,225,W99-0505,C94-2113,0,Towards a Meaning-Full Comparison of Lexieal R...,Towards Meaning-Full Comparison Lexieal Resour...,Towards MeaningFull Comparison Lexieal Resourc...


In [ ]:
# sizes of texts 
clean_dataset['w/o stopwords or tags'].apply(len).sort_values()

In [ ]:
#try removing thousand word lines from dataset to make training easier
smaller_clean_dataset = clean_dataset.loc[clean_dataset['w/o stopwords or tags'].apply(len) < 1000]
smaller_clean_dataset['w/o stopwords or tags'].apply(len).sort_values()

In [ ]:
#redefine maxlen as length of new longest sentence
maxLen = max(smaller_clean_dataset['w/o stopwords or tags'].apply(len))

## Splitting sentences

In [ ]:
# split into test and training data
X_train, X_test,Y_train, Y_test = train_test_split(smaller_clean_dataset['w/o stopwords or tags'], smaller_clean_dataset['label'], test_size=0.2, random_state = 45)

In [ ]:
# create list of unique words in sentences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

#creates dictionary of each {word: index}
words_to_index = tokenizer.word_index
words_to_index

In [ ]:
#function to read GloCe Vector file
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [ ]:
# read in GloVe vector from Google Drive (premade mapping of words to be used for sentiment analysis)
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/UMD - senior year/spring 2022/439D/project/glove.6B.50d.txt')

In [ ]:
from tensorflow.keras.layers import Embedding


# create embedding matrix (all words in GloVe vector assigned to correct value matrix, all others assigend to 0 vector)
vocab_len = len(words_to_index) + 1
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))


#get glove coordinates of words that are in BOTH glove list and in training data sentences 
for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [ ]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [ ]:
# create model architecture
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  #X = Dense(1, activation='sigmoid')(X)

  X = Dense(3, activation='softmax')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
#tokenizer converts words in the sentence into its corresponding indexe 
print(X_train.iloc[-1])  
print(tokenizer.texts_to_sequences(X_train)[-1])
print('the word since, the first word of the last sentence in the dataset has index: ', words_to_index['regardless'])

Regardless whether takes form dictionaries Lesk  Guthrie et al  Dagan Itai Schwall  Karov Edelman  thesauri Yarowsky  Walker Amsler  bilingual corpora Brown et al  Church Gale  handlabeled training sets Hearst  Leacock Towell Voorhees  Niwa Nitta  Bruce Wiebe  providing information sense definitions can considerable burden
[3871, 472, 748, 229, 484, 906, 807, 2, 1, 125, 1738, 3872, 3873, 3874, 674, 154, 3875, 3876, 568, 99, 26, 2, 1, 3, 269, 3877, 35, 265, 1833, 709, 1834, 980, 1771, 1772, 1835, 1153, 3878, 5, 140, 599, 17, 584, 3879]
the word since, the first word of the last sentence in the dataset has index:  3871


In [ ]:
#convert training data sentence words into its corresponding indexes
X_train_indices = tokenizer.texts_to_sequences(X_train)

#pad sentences of indexes so all same length
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
print('the shape of the new training data is 796 samples, each with 835 words (bc of padding): ', X_train_indices.shape)

the shape of the new training data is 796 samples, each with 835 words (bc of padding):  (796, 835)


In [ ]:
#train the model
model = imdb_rating((maxLen,))

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15


ValueError: ignored

In [ ]:
#convert the testing data sentence words into its corresponding indexes
X_test_indices = tokenizer.texts_to_sequences(X_test)

#pad sentences of indexes so all same length
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
print('the shape of the new training data is 200 samples, each with 835 words (bc of padding): ', X_test_indices.shape)


model.evaluate(X_test_indices, Y_test)

the shape of the new training data is 200 samples, each with 835 words (bc of padding):  (200, 835)


ValueError: ignored

In [ ]:
my_predictions = model.predict(X_test_indices)

In [ ]:
my_predictions

array([[0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476176],
       [0.05476176],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476176],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.05476177],
       [0.05476177],
       [0.05476176],
       [0.054

In [ ]:
smaller_clear_dataset['label'].unique()

array([ 0,  1, -1])